In [ ]:
###      THIS NOTEBOOK IS BEING REVISED        ###

In [9]:
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot as plt
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack

In [2]:
def make_D(in_shape = (28, 28, 1)):

    model = Sequential([
        Conv2D(64, (3,3), strides = (2,2), padding = 'same', input_shape = in_shape),
        LeakyReLU(alpha = 0.2),
        Dropout(0.4),
        Conv2D(64, (3,3), strides = (2,2), padding = 'same'),
        LeakyReLU(alpha = 0.2),
        Dropout(0.4),
        Flatten(),
        Dense(1, activation = 'sigmoid')
    ])

    opt = Adam(learning_rate = 0.0002, beta_1 = 0.5)

    model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = (['accuracy']))

    return model

In [3]:
def make_G(latent_dims):

    n_nodes = 128 * 7 * 7 # 128 images of 7x7
    model = Sequential([
        Dense(n_nodes, input_dim = latent_dims),
        LeakyReLU(alpha = 0.2),
        Reshape((7, 7, 128)),
        Conv2DTranspose(128, (4, 4), strides = (2,2), padding = 'same'),
        LeakyReLU(alpha = 0.2),
        Conv2DTranspose(128, (4, 4), strides = (2,2), padding = 'same'),
        LeakyReLU(alpha = 0.2),
        Conv2D(1, (7,7),  activation = 'sigmoid', padding = 'same')
    ])

    return model

In [4]:
def make_GAN(G, D):

    D.trainable = False

    gan = Sequential()
    gan.add(G)
    gan.add(D)

    opt = Adam(learning_rate = 0.0002, beta_1 = 0.5)

    gan.compile(loss = 'binary_crossentropy', optimizer = opt)

    return gan

In [5]:
def load_real_samples():

    (Xt, _), (_, _) = load_data()

    X = expand_dims(Xt, axis = -1)

    X = X.astype('float32')
    X = X / 255.0

    return X

In [6]:
def generate_real_samples(dataset, n_samples):

    i = randint(0, dataset.shape[0], n_samples)
    X = dataset[i]
    y = ones((n_samples, 1))
    return X, y

In [7]:
def generate_latent_points(latent_dims, n_samples):

    X = randn(latent_dims * n_samples)
    X = X.reshape((n_samples, latent_dims))

    return X

In [8]:
def generate_fake_samples(G, latent_dims, n_samples):

    L = generate_latent_points(latent_dims, n_samples)

    X = G.predict(L, verbose = None)

    y = zeros((n_samples, 1))

    return X, y

In [10]:
def save_images(examples, epoch, n = 10):

    for i in range(n * n):

        plt.subplot(n, n, i + 1)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap = 'gray')

    filename = 'gen_plot_%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()

In [12]:
def summarize_performance(dataset, D, G, epoch, latent_dim, n_samples = 100):

    X_real, y_real = generate_real_samples(dataset, n_samples)

    _, acc_real = D.evaluate(X_real, y_real, verbose = 0)

    X_fake, y_fake = generate_fake_samples(G, latent_dim, n_samples)

    _, acc_fake = D.evaluate(X_fake, y_fake, verbose = 0)

    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))

    #print(f'Acc Real : {acc_real} -- Acc Fake : {acc_fake}')

    save_images(X_fake, epoch)

    filename = 'gen_mnist_%03d.h5' % (epoch+1)
    G.save(filename)

In [13]:
def train_GAN(G, D, GAN, dataset, latent_dim, n_epochs = 100, batch_size = 256):

    batch_p_epoch = dataset.shape[0] // n_epochs
    half_batch = batch_size // 2

    for epoch in range(n_epochs):
        for batch in range(batch_p_epoch):

            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(G, latent_dim, half_batch)

            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))

            d_loss, _ = D.train_on_batch(X, y)

            X_gan = generate_latent_points(latent_dim, batch_size)

            y_gan = ones((batch_size, 1))

            g_loss = GAN.train_on_batch(X_gan, y_gan)

            print('>%d, %d/%d, d=%.3f, g=%.3f' % (epoch+1, batch+1, batch_p_epoch, d_loss, g_loss))

            #print(f'epoch :{epoch+1} - batch : {batch+1}/{batch_p_epoch} - d_loss : {d_loss} - g_loss : {g_loss}')

        if (epoch+1) % 10 == 0 :
            summarize_performance(dataset, D, G, epoch, latent_dim)


In [ ]:
latent_dims = 100
D = make_D()
G = make_G(latent_dims)
GAN = make_GAN(G, D)

dataset = load_real_samples()

train_GAN(G, D, GAN, dataset, latent_dims)